In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSD%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFNSD%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_temperature_interface(tag=interface_tag, init_values=T_bot)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=300.0)
tempS = InterfaceData(size=inodes, value=T_bot)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 500
t = 0.0
avg_pc_iterations = 0.0
flog = open('FNSD%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Dirichlet setting, study case: %i\n' % k)
flog.close()
flog = open('FNSD%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    # mapping
    fluxF.curr[:] = fluxS.curr[s2f]
    
    solverF.put_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=fluxF.curr
    )
    
    while True:
        fluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface temperature
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        tempS.curr[:] = tempF.curr[f2s]
        
        solverS.update_temperature_interface(interface_tag, tempS.curr)
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface flux
        fluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        fluxF.curr[:] = fluxS.curr[s2f]
        
        fluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(fluxF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to solid then restore
            if use_relax:
                under_relax.determine_omega(fluxF)
                under_relax.update_solution(fluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=fluxF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    
flog.close()

step=0, pc_iterations=20.
step=1, pc_iterations=17.
step=2, pc_iterations=14.
step=3, pc_iterations=16.
step=4, pc_iterations=15.
step=5, pc_iterations=16.
step=6, pc_iterations=14.
step=7, pc_iterations=16.
step=8, pc_iterations=15.
step=9, pc_iterations=15.
step=10, pc_iterations=15.
step=11, pc_iterations=14.
step=12, pc_iterations=14.
step=13, pc_iterations=14.
step=14, pc_iterations=14.
step=15, pc_iterations=14.
step=16, pc_iterations=14.
step=17, pc_iterations=14.
step=18, pc_iterations=13.
step=19, pc_iterations=15.
step=20, pc_iterations=14.
step=21, pc_iterations=15.
step=22, pc_iterations=14.
step=23, pc_iterations=15.
step=24, pc_iterations=14.
step=25, pc_iterations=15.
step=26, pc_iterations=14.
step=27, pc_iterations=15.
step=28, pc_iterations=14.
step=29, pc_iterations=14.
step=30, pc_iterations=14.
step=31, pc_iterations=14.
step=32, pc_iterations=14.
step=33, pc_iterations=14.
step=34, pc_iterations=14.
step=35, pc_iterations=14.
step=36, pc_iterations=14.
step=37, pc

step=297, pc_iterations=14.
step=298, pc_iterations=13.
step=299, pc_iterations=14.
step=300, pc_iterations=13.
step=301, pc_iterations=14.
step=302, pc_iterations=13.
step=303, pc_iterations=14.
step=304, pc_iterations=15.
step=305, pc_iterations=15.
step=306, pc_iterations=15.
step=307, pc_iterations=14.
step=308, pc_iterations=13.
step=309, pc_iterations=13.
step=310, pc_iterations=14.
step=311, pc_iterations=14.
step=312, pc_iterations=14.
step=313, pc_iterations=10.
step=314, pc_iterations=12.
step=315, pc_iterations=12.
step=316, pc_iterations=14.
step=317, pc_iterations=14.
step=318, pc_iterations=14.
step=319, pc_iterations=15.
step=320, pc_iterations=14.
step=321, pc_iterations=14.
step=322, pc_iterations=13.
step=323, pc_iterations=12.
step=324, pc_iterations=13.
step=325, pc_iterations=12.
step=326, pc_iterations=13.
step=327, pc_iterations=9.
step=328, pc_iterations=14.
step=329, pc_iterations=14.
step=330, pc_iterations=14.
step=331, pc_iterations=14.
step=332, pc_iteratio

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFNSD%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.9245013  0.93081567 0.93588092 0.93875971 0.94122241 0.94349557
 0.9455119  0.94735014 0.94912725 0.95071865 0.95221615 0.95361185
 0.95497028 0.95623887 0.95745446 0.95859268 0.95970557 0.96075832
 0.96175915 0.96273439 0.9636581  0.96455393 0.96541327 0.96623292
 0.96703287 0.96780095 0.96854615 0.96925129 0.96992716 0.97058714
 0.97122596 0.97183689 0.97243331 0.97301107 0.97356561 0.97409881
 0.97462171 0.9751263  0.97561747 0.97608499 0.97653633 0.97697389
 0.97740151 0.97781517 0.97821586 0.97860345 0.97898158 0.9793438
 0.97969563 0.9800421  0.98037633 0.98069637 0.98100909 0.98131336
 0.98160353 0.98188815 0.98216016 0.98242677 0.98268514 0.9829316
 0.98317029 0.98340473 0.98362993 0.98384378 0.98405018 0.98425239
 0.98444075 0.98462471 0.98479715 0.98496271 0.98512272 0.98527066
 0.98541129 0.9855469  0.98567235 0.98578653 0.98589173 0.98599057
 0.98607438 0.98615189 0.98621505 0.98626932 0.98631279 0.98634496
 0.9863622  0.98636503 0.98635503 0.98632609 0.98628025 0.986216